In [2]:
!pip install mesa
from mesa import Agent, Model 
from mesa.space import MultiGrid
from mesa.time import SimultaneousActivation
from mesa.datacollection import DataCollector
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128
import numpy as np
import pandas as pd
import time
import datetime
import random

In [4]:
def get_grid(model):
    '''
    Esta es una función auxiliar que nos permite guardar el grid para cada uno de los agentes.
    param model: El modelo del cual optener el grid.
    return una matriz con la información del grid del agente.
    '''
    grid = np.zeros((model.grid.width, model.grid.height))
    for cell in model.grid.coord_iter():
        cell_content, x, y = cell
        for obj in cell_content:
          if isinstance(cell_content, RobotLimpieza):
            grid[x][x] = 2
          elif isinstance(cell_content, Celda):
            grid[x][y] = obj.estado
    return grid

class RobotLimpieza(Agent):
  '''
  En esta secicón se define la maquina de estados que está basada en el diagrama
  que fue elaborado en el software yED, en el cual se define el comportamiento
  del agente mediante una secuencia de condiciones simples.
  '''
  def __init__(self, unique_id, model):
    super().__init__(unique_id,model)
    self.sig_pos = None
  
  def step(self):
    vecinos = self.model.grid.get_neighbors(
            self.pos,
            moore=True,
            include_center=True)
    
    #Maquina de estados
    for vecino in vecinos:
      if isinstance(vecino, Celda) and self.pos == vecino.pos:
        if vecino.estado == 1:
        #Limpieza 
          vecino.sig_estado = 0
          self.sig_pos = self.pos
        else:
          vecinos_2 = self.model.grid.get_neighbors(
            self.pos,
            moore=True,
            include_center=False)
          vecino.sig_estado = 0
          self.sig_pos = self.random.choice(vecinos_2)
        break

    def advance(self):
      vecinos  = self.model.grid.get_neighbors(
            self.pos,
            moore=True,
            include_center=True)
      for vecino in vecinos:
        if isinstance(vecino, Celda) and self.pos == vecino.pos:
          vecino.estado = vecino.sig_estado
      self.model.grid.move_agent(self, self.sig_pos)



class Celda(Agent):
  '''
  Aqio se definen los estados de las celdas
  '''
  #1 = Sucio
  #0 = Limpio
  def __init__(self, unique_id, model,estado):
    super().__init__(unique_id, model)
    self.pos = unique_id
    self.estado = estado
    self.sig_estado = None

class Habitacion(Model): 
  '''
  Aqui se define y se realiza la asignacion de celdas sucias y limpias en el 
  grid que en este caso es m*n lo cual nos da el tamaño total
  '''
  def __init__(self, m, n, num_agentes, por_celdas_sucias):
    self.num_agentes = num_agentes
    self.por_celdas_sucias = por_celdas_sucias
    self.por_celdas_limpias = 1 - por_celdas_sucias
    self.grid = MultiGrid(m, n, False)
    self.schedule = SimultaneousActivation(self)
    
    #Asignacion de celdas limpias y sucias
    num_celdas_sucias = int (m*n * por_celdas_sucias)
    for (content, x, y) in self.grid.coord_iter():
      num = random.randint(0,1)

      if num ==1 and num_celdas_sucias >0:
        a = Celda((x,y), self,1)
        num_celdas_sucias -=1
      else:
        a = Celda((x,y), self, 0)
      self.grid.place_agent(a, (x, y))
      self.schedule.add(a)
      
      #Asignacion de agentes 
      
    for id in range (num_agentes):
      r = RobotLimpieza(id, self)
      self.grid.place_agent(r, (1, 1))
      self.schedule.add(r)

    self.datacollector = DataCollector(
        model_reporters= {"Grid": get_grid})
      
    def step(self):
      self.datacollector.collect(self)
      self.schedule.step()





In [6]:
#Variables de tamaño de grid
m = 15
n =10
#Numero de agentes y el porcentaje de celdas sucias
num_agentes = 5
por_celdas_sucias = 0.6
#Tiempo máximo
tiempo_maximo = 0.5

model =Habitacion(m, n, num_agentes, por_celdas_sucias)
start_time = time.time()
while ((time.time() - start_time)<tiempo_maximo):
  model.step()
print('Tiempo de ejecución:', str(datetime.timedelta(seconds=(time.time() - start_time))))

Tiempo de ejecución: 0:00:00.500113


# Seccion 3
Creacion del grid 

In [7]:
all_grid = model.datacollector.get_model_vars_dataframe()

# Seccion 4

Por alguna razón el código no quiere hacer el caputure porque marca un out of bounds, traté de resolver ese problema, pero no pude encontrar el error

In [8]:
%%capture

fig, axs = plt.subplots(figsize=(m,n))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap=plt.cm.binary)
def animate(i):
  patch.set_data(all_grid.iloc[i][0])
anim = animation.FuncAnimation(fig, animate, frames=len(all_grid))


IndexError: single positional indexer is out-of-bounds